In [12]:
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
import numpy as np
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

In [13]:
corpus = pd.read_csv('snappfood_comments_preprocessed.csv')

In [27]:
corpus.head()

,Unnamed: 0,comment,label,label_id,Cleaned
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0,قرار بود 1 ساعته برسه ولی نیم ساعت زودتر از مو...
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0,عالی بود همه چه درست و به اندازه و کیفیت خوب، ...
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0,شیرینی وانیلی فقط یک مدل بود


In [14]:
labels = corpus['label_id'].values

In [15]:
X = corpus['Cleaned']
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)
X=tokenizer.texts_to_sequences(X)
X=pad_sequences(X,maxlen=100)

In [16]:
X.shape

(69480, 100)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42) 

In [18]:
y_train_cat = np_utils.to_categorical(y_train, 2)
y_test_cat = np_utils.to_categorical(y_test, 2)

In [19]:
from tensorflow.keras import layers

In [20]:
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=20000, output_dim=32, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [21]:
rnn_model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [22]:
rnn_model.fit(X_train, y_train_cat, epochs=3,
              validation_data=(X_test, y_test_cat),
              validation_steps=30)

Epoch 1/3
1737/1737 [==============================] - 855s 485ms/step - loss: 0.4108 - accuracy: 0.8082 - val_loss: 0.3569 - val_accuracy: 0.8501
Epoch 2/3
1737/1737 [==============================] - 882s 508ms/step - loss: 0.3277 - accuracy: 0.8628 - val_loss: 0.3507 - val_accuracy: 0.8533
Epoch 3/3
1737/1737 [==============================] - 930s 535ms/step - loss: 0.3022 - accuracy: 0.8756 - val_loss: 0.3472 - val_accuracy: 0.8526


In [23]:
y_test_pred = rnn_model.predict(X_test)
y_test_predclass = np.argmax(y_test_pred, axis=1)
y_trian_pred = rnn_model.predict(X_train)
y_train_predclass = np.argmax(y_trian_pred, axis=1)

1737/1737 [==============================] - 433s 249ms/step


In [24]:
from sklearn.metrics import accuracy_score,classification_report
print ("nDeep Neural Network - Test accuracy:",(round(accuracy_score(y_test, y_test_predclass),4)*100))
print ("nDeep Neural Network - Train accuracy:",(round(accuracy_score(y_train, y_train_predclass),4)*100))

nDeep Neural Network - Test accuracy: 85.26
nDeep Neural Network - Train accuracy: 89.31


In [59]:
X_pred=tokenizer.texts_to_sequences(['غذا خیلی گرم بود'])
X_pred=pad_sequences(X_pred,maxlen=100)

In [60]:
rnn_model.predict(X_pred)

1/1 [==============================] - 0s 48ms/step


array([[0.67157423, 0.32842582]], dtype=float32)